In [1]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4" # specify which GPU(s) to be used
bm.disable_gpu_memory_preallocation()
bm.set_platform('gpu')

/home/spirit/anaconda3/envs/cy_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def genetic_algorithm(A, b, population_size=500, generations=1000, mutation_rate=0.1, crossover_rate=0.7, sparsity=10):
    """
    Genetic Algorithm for Sparse Coding
    """
    m, n = A.shape

    # Initialize population
    population = np.random.randn(population_size, n)
    for i in range(population_size):
        # Enforce sparsity by zeroing out random coefficients
        zero_indices = np.random.choice(n, n - sparsity, replace=False)
        population[i, zero_indices] = 0

    for gen in range(generations):
        # Evaluate fitness (negative reconstruction error plus sparsity penalty)
        fitness = -np.linalg.norm(A @ population.T - b.reshape(-1, 1), axis=0) - 0.01 * np.count_nonzero(population, axis=1)

        # Selection (roulette wheel selection)
        fitness_shifted = fitness - fitness.min() + 1e-6  # Ensure positive fitness
        probabilities = fitness_shifted / fitness_shifted.sum()
        indices = np.random.choice(population_size, size=population_size, p=probabilities)
        selected_population = population[indices]

        # Crossover
        new_population = []
        for i in range(0, population_size, 2):
            parent1 = selected_population[i]
            parent2 = selected_population[(i+1) % population_size]
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(1, n)
                child1 = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
                child2 = np.hstack((parent2[:crossover_point], parent1[crossover_point:]))
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            new_population.extend([child1, child2])

        # Mutation
        for individual in new_population:
            if np.random.rand() < mutation_rate:
                mutation_indices = np.random.choice(n, sparsity // 2, replace=False)
                individual[mutation_indices] += np.random.randn(mutation_indices.size)

        # Enforce sparsity
        for individual in new_population:
            sorted_indices = np.argsort(np.abs(individual))
            individual[sorted_indices[:-sparsity]] = 0

        population = np.array(new_population)

        # Optional: Print best fitness every few generations
        if (gen + 1) % 10 == 0:
            best_fitness = -fitness.max()
            print(f"Generation {gen + 1}, Best Fitness: {best_fitness:.4f}")

    # Return the best individual
    fitness = -np.linalg.norm(A @ population.T - b.reshape(-1, 1), axis=0) - 0.01 * np.count_nonzero(population, axis=1)
    best_index = np.argmax(fitness)
    best_solution = population[best_index]
    return best_solution


def differential_evolution_sparse(A, b, lam, pop_size=50, max_iter=1000, F=0.8, CR=0.9):
    """
    Differential Evolution for Sparse Coding
    """
    m, n = A.shape
    # Objective function
    def objective(x):
        return np.linalg.norm(A @ x - b) ** 2 + lam * np.linalg.norm(x, 1)

    # Initialize population
    pop = np.random.randn(pop_size, n)
    # Evaluate initial population
    fitness = np.array([objective(ind) for ind in pop])

    for iter in range(max_iter):
        for i in range(pop_size):
            # Mutation
            idxs = [idx for idx in range(pop_size) if idx != i]
            a, b_ind, c = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = a + F * (b_ind - c)
            # Crossover
            cross_points = np.random.rand(n) < CR
            if not np.any(cross_points):
                cross_points[np.random.randint(0, n)] = True
            trial = np.where(cross_points, mutant, pop[i])
            # Selection
            f = objective(trial)
            if f < fitness[i]:
                pop[i] = trial
                fitness[i] = f

        # Optional: Print progress
        if (iter + 1) % 100 == 0:
            print(f"Iteration {iter + 1}, Best Fitness: {fitness.min():.4f}")

    # Return the best individual
    best_idx = np.argmin(fitness)
    best_solution = pop[best_idx]
    return best_solution

def particle_swarm_optimization_sparse(A, b, lam, pop_size=30, max_iter=1000, w=0.7, c1=1.5, c2=1.5):
    """
    Particle Swarm Optimization for Sparse Coding
    """
    m, n = A.shape
    # Objective function
    def objective(x):
        return np.linalg.norm(A @ x - b) ** 2 + lam * np.linalg.norm(x, 1)

    # Initialize particles
    x = np.random.randn(pop_size, n)
    v = np.zeros((pop_size, n))
    p_best = x.copy()
    p_best_val = np.array([objective(ind) for ind in x])
    g_best_idx = np.argmin(p_best_val)
    g_best = p_best[g_best_idx].copy()

    for iter in range(max_iter):
        for i in range(pop_size):
            r1 = np.random.rand(n)
            r2 = np.random.rand(n)
            # Update velocity
            v[i] = w * v[i] + c1 * r1 * (p_best[i] - x[i]) + c2 * r2 * (g_best - x[i])
            # Update position
            x[i] = x[i] + v[i]
            # Apply soft thresholding for sparsity
            x[i] = np.sign(x[i]) * np.maximum(np.abs(x[i]) - lam * 0.1, 0)
            # Evaluate
            f = objective(x[i])
            # Update personal best
            if f < p_best_val[i]:
                p_best[i] = x[i].copy()
                p_best_val[i] = f
                # Update global best
                if f < p_best_val[g_best_idx]:
                    g_best = x[i].copy()
                    g_best_idx = i

        # Optional: Print progress
        if (iter + 1) % 100 == 0:
            print(f"Iteration {iter + 1}, Best Fitness: {p_best_val[g_best_idx]:.4f}")
    return g_best

m, n = 5000, 10000
A = np.random.randn(m, n)
sparsity_value = int(0.1 * n)
x_true = np.zeros(n)
non_zero_indices = np.random.choice(n, sparsity_value, replace=False)
x_true[non_zero_indices] = np.random.randn(sparsity_value)
b = A @ x_true + 0.1 * np.random.randn(m)

x_ga = genetic_algorithm(A, b, sparsity=sparsity_value)
print("Genetic Algorithm Solution:")
print(x_ga)

Generation 10, Best Fitness: 2979.6792
Generation 20, Best Fitness: 2936.1721
Generation 30, Best Fitness: 2915.5439
Generation 40, Best Fitness: 2895.5864
Generation 50, Best Fitness: 2861.0172
Generation 60, Best Fitness: 2896.3543
Generation 70, Best Fitness: 2869.5685
Generation 80, Best Fitness: 2843.6312
Generation 90, Best Fitness: 2838.0723
Generation 100, Best Fitness: 2836.3082
Generation 110, Best Fitness: 2828.5495
Generation 120, Best Fitness: 2829.9187
Generation 130, Best Fitness: 2816.4858
Generation 140, Best Fitness: 2795.5762
Generation 150, Best Fitness: 2801.0294
Generation 160, Best Fitness: 2780.2300
Generation 170, Best Fitness: 2783.5249
Generation 180, Best Fitness: 2781.5139
Generation 190, Best Fitness: 2758.9276
Generation 200, Best Fitness: 2786.6119
Generation 210, Best Fitness: 2777.4903
Generation 220, Best Fitness: 2769.8142
Generation 230, Best Fitness: 2769.5303
Generation 240, Best Fitness: 2776.6692
Generation 250, Best Fitness: 2765.4581
Generatio